In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import argparse
import sys
import tempfile

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


In [2]:
def deepnn(x):
    """deepnn builds the graph for a deep net for classifying digits.
    Args:
    x: an input tensor with the dimensions (N_examples, 784), where 784 is the
    number of pixels in a standard MNIST image.
    Returns:
    A tuple (y, keep_prob). y is a tensor of shape (N_examples, 10), with values
    equal to the logits of classifying the digit into one of 10 classes (the
    digits 0-9). keep_prob is a scalar placeholder for the probability of
    dropout.
    """
    # Reshape to use within a convolutional neural net.
    # Last dimension is for "features" - there is only one here, since images are
    # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
    with tf.name_scope('reshape'):
        x_image = tf.reshape(x, [-1, 28, 28, 1])

    # First convolutional layer - maps one grayscale image to 32 feature maps.
    with tf.name_scope('conv1'):
        W_conv1 = weight_variable([5, 5, 1, 32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

    # Pooling layer - downsamples by 2X.
    with tf.name_scope('pool1'):
        h_pool1 = max_pool_2x2(h_conv1)

    # Second convolutional layer -- maps 32 feature maps to 64.
    with tf.name_scope('conv2'):
        W_conv2 = weight_variable([5, 5, 32, 64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

    # Second pooling layer.
    with tf.name_scope('pool2'):
        h_pool2 = max_pool_2x2(h_conv2)

    # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
    # is down to 7x7x64 feature maps -- maps this to 1024 features.
    with tf.name_scope('fc1'):
        W_fc1 = weight_variable([7 * 7 * 64, 1024])
        b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    # Dropout - controls the complexity of the model, prevents co-adaptation of
    # features.
    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    # Map the 1024 features to 10 classes, one for each digit
    with tf.name_scope('fc2'):
        W_fc2 = weight_variable([1024, 10])
        b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    return y_conv, keep_prob

In [3]:
def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


In [4]:

from tensorflow.examples.tutorials.mnist import input_data

    # Import Fashion MNIST
    


def main(_):
    # Import data
#     mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)
    print ("importing fashion mnist")
    mnist = input_data.read_data_sets('input/data', one_hot=True)


    # Create the model
    x = tf.placeholder(tf.float32, [None, 784])

    # Define loss and optimizer
    y_ = tf.placeholder(tf.float32, [None, 10])

    # Build the graph for the deep net
    y_conv, keep_prob = deepnn(x)

    with tf.name_scope('loss'):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,
                                                            logits=y_conv)
    cross_entropy = tf.reduce_mean(cross_entropy)

    with tf.name_scope('adam_optimizer'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)

    graph_location = tempfile.mkdtemp()
    print('Saving graph to: %s' % graph_location)
    train_writer = tf.summary.FileWriter(graph_location)
    train_writer.add_graph(tf.get_default_graph())

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(40001):
            batch = mnist.train.next_batch(50)
            if i % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                    x: batch[0], y_: batch[1], keep_prob: 1.0})
                print('step %d, training accuracy %g' % (i, train_accuracy))
                train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
            
            if i % 1000 == 0:
                print('test accuracy %g' % accuracy.eval(feed_dict={
                    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

        print('test accuracy %g' % accuracy.eval(feed_dict={
            x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [ ]:
    #    Hidden state
    get_last_layer_output = K.function([model.layers[0].input, K.learning_phase()],
                                  [model.layers[7].output])

    #    get_last_layer_output = K.function([model.layers[0].input],
    #                                  [model.layers[4].output])

    # output in train mode = 0
    #    layer_output = np.array(get_last_layer_output([X_train[0:1200], 0])[0])

#     output in train mode = 0
    start = 0
    increment = 100
    flag = 1
    while start+increment <= len(X_train):
        if flag:
            layer_output = get_last_layer_output([X_train[start:start+increment], 0])[0]
            flag = 0
        else:
            layer_output = np.concatenate((layer_output, get_last_layer_output([X_train[start:start+increment], 0])[0]))
        start += increment
    if start != len(X_train):
        layer_output = np.concatenate((layer_output, get_last_layer_output([X_train[start:len(X_train)], 0])[0]))

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', type=str,
                  default='/tmp/tensorflow/mnist/input_data',
                  help='Directory for storing input data')
FLAGS, unparsed = parser.parse_known_args()
tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

importing fashion mnist
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting input/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting input/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting input/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting input/data/t10k-labels-idx1-ubyte.gz
Saving graph to: /var/folders/1w/4zq3xcqn6s96fd3ksy_htvgr0000gn/T/tmpUQhumP
step 0, training accuracy 0.06
test accuracy 0.0703
step 100, training accuracy 0.04
step 200, training accuracy 0.08
step 300, training accuracy 0.1
step 400, training accuracy 0.06
step 500, training accuracy 0.08
step 600, training accuracy 0.16
step 700, training accuracy 0.14
step 800, training accuracy 0.22
step 900, training accuracy 0.16
step 1000, training accuracy 0.28
test accuracy 0.2273
step 1100, training accuracy 0.2
step 1200, tr

step 21000, training accuracy 0.92
test accuracy 0.9086
step 21100, training accuracy 0.9
step 21200, training accuracy 0.9
step 21300, training accuracy 0.94
step 21400, training accuracy 0.96
step 21500, training accuracy 0.86
step 21600, training accuracy 0.88
step 21700, training accuracy 0.92
step 21800, training accuracy 0.9
step 21900, training accuracy 0.86
step 22000, training accuracy 0.92
test accuracy 0.9122
step 22100, training accuracy 0.92
step 22200, training accuracy 0.92
step 22300, training accuracy 0.96
step 22400, training accuracy 0.86
step 22500, training accuracy 0.92
step 22600, training accuracy 0.86
step 22700, training accuracy 0.9
step 22800, training accuracy 0.92
step 22900, training accuracy 0.92
step 23000, training accuracy 0.9
test accuracy 0.9147
step 23100, training accuracy 0.82
step 23200, training accuracy 0.9
step 23300, training accuracy 0.86
step 23400, training accuracy 0.88
step 23500, training accuracy 1
step 23600, training accuracy 0.94
s

SystemExit: 

/Users/shubhi/anaconda/envs/py2_latest/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [45]:
print ("this function will take the output of hidden layers and bin them into 30 bins (by default) from -1 to 1 and return the means and probability of each bin ")
def bin_hidden_layers(T, num_bins = 30, beg = -1, end =1 ):
    import numpy as np
    data = np.array(T)
    bins = numpy.linspace(beg, end, num_bins)
    digitized = numpy.digitize(data, bins)
    print (digitized)
    bin_means = [data[digitized == i].mean() for i in range(1, len(bins) +1)]
    print (bin_means)
    
    binned_data = digitized -1

    from collections import Counter
    counter = Counter(binned_data)

    binned_mean = []
    binned_probability = []
    for class1 in counter :
        prob = counter[class1]/ len(binned_data)
        bin_mean = bin_means[class1]
#         print (class1 )
#         print (prob)
#         print (bin_mean)
        binned_mean.append(bin_mean)
        binned_probability.append(prob)

    
    return (binned_mean , binned_probability)
    

this function will take the output of hidden layers and bin them into 30 bins (by default) from -1 to 1 and return the means and probability of each bin 


In [48]:
print ("test function for binning ")
import numpy 
T  = [-1, -0.7, -0.4, -0.3, -0.2,-1, -0.7, -0.4, -0.3, -0.2,-1, -0.7, -0.4, 
      -0.3, -0.2,-1, -0.7, -0.4, -0.3, -0.2, 1, 0, 0.4, 0.3, 0.2, 0.1, 0.5, 0.55, 0.56 ]
T = numpy.array(T)
print (T)
binned_mean , binned_probability = bin_hidden_layers(T, num_bins=4)
print (binned_mean)
print (binned_probability)


test function for binning 
[-1.   -0.7  -0.4  -0.3  -0.2  -1.   -0.7  -0.4  -0.3  -0.2  -1.   -0.7
 -0.4  -0.3  -0.2  -1.   -0.7  -0.4  -0.3  -0.2   1.    0.    0.4   0.3
  0.2   0.1   0.5   0.55  0.56]
[1 1 1 2 2 1 1 1 2 2 1 1 1 2 2 1 1 1 2 2 4 2 3 2 2 2 3 3 3]
[-0.70000000000000007, -0.11666666666666665, 0.50250000000000006, 1.0]
[-0.70000000000000007, -0.11666666666666665, 0.50250000000000006, 1.0]
[0.41379310344827586, 0.41379310344827586, 0.13793103448275862, 0.034482758620689655]
